In [1]:
import nest_asyncio
nest_asyncio.apply()

In [9]:
import pandas as pd
TelegramDF = pd.read_csv('TelegramList.csv', sep=';')

In [6]:
TelegramDF

,Название СМИ,Ссылка
0,Nexta live,https://t.me/nexta_live
1,RESSENTIMENT,https://t.me/ressentiment_channel
2,Пивоваров (Редакция),https://t.me/redakciya_channel
3,Varlamov News,https://t.me/varlamov_news
4,Медуза — LIVE,https://t.me/meduzalive
5,СИЛОВИКИ 🇷🇺,https://t.me/SIL0VIKI
6,Постовой,https://t.me/Postovo
7,Милитарист,https://t.me/infantmilitario
8,brief ®️незыгарь,https://t.me/russicabrief
9,Раньше всех. Ну почти.,https://t.me/bbbreaking


In [11]:
import configparser
import json

from telethon.sync import TelegramClient
from telethon import connection

# для корректного переноса времени сообщений в json
from datetime import date, datetime

# классы для работы с каналами
from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch

# класс для работы с сообщениями
from telethon.tl.functions.messages import GetHistoryRequest

# Считываем учетные данные
config = configparser.ConfigParser()
config.read("config.ini")

# Присваиваем значения внутренним переменным
api_id   = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']
username = config['Telegram']['username']

client = TelegramClient(username, api_id, api_hash)

client.start()

async def dump_all_messages(channel, channel_name):
	"""Записывает json-файл с информацией о всех сообщениях канала/чата"""
	offset_msg = 0    # номер записи, с которой начинается считывание
	limit_msg = 100   # максимальное число записей, передаваемых за один раз

	all_messages = []   # список всех сообщений
	total_messages = 0
	total_count_limit = 1000  # поменяйте это значение, если вам нужны не все сообщения

	class DateTimeEncoder(json.JSONEncoder):
		'''Класс для сериализации записи дат в JSON'''
		def default(self, o):
			if isinstance(o, datetime):
				return o.isoformat()
			if isinstance(o, bytes):
				return list(o)
			return json.JSONEncoder.default(self, o)

	while True:
		history = await client(GetHistoryRequest(
			peer=channel,
			offset_id=offset_msg,
			offset_date=None, add_offset=0,
			limit=limit_msg, max_id=0, min_id=0,
			hash=0))
		if not history.messages:
			break
		messages = history.messages
		for message in messages:
			all_messages.append(message.to_dict())
		offset_msg = messages[len(messages) - 1].id
		total_messages = len(all_messages)
		if total_count_limit != 0 and total_messages >= total_count_limit:
			break

	with open(channel_name+'.json', 'w', encoding='utf8') as outfile:
		 json.dump(all_messages, outfile, ensure_ascii=False, cls=DateTimeEncoder)


async def main(link, channel_name):
	url = link
	channel = await client.get_entity(url)
	await dump_all_messages(channel, channel_name)


async with client:
    for i in range(TelegramDF['Название СМИ'].size):
        link = TelegramDF.loc[i, 'Ссылка']
        channel_name = TelegramDF.loc[i, 'Название СМИ']
        try:
            client.loop.run_until_complete(main(link, channel_name))
        except:
            pass

/var/folders/tm/wz5tvn7951s2f6hclbtzc9gw0000gn/T/ipykernel_29212/1032581251.py:28: RuntimeWarning: coroutine 'AuthMethods._start' was never awaited
  client.start()
